In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import skimage
plt.rcParams.update({'font.size': 18})

from skimage import data
from skimage import io
from skimage import img_as_float
from skimage import color
from skimage import filters
from skimage import measure
from skimage import segmentation
from scipy.ndimage import gaussian_filter

from PIL import Image
import PIL.TiffTags
from PIL.TiffTags import TAGS
Image.MAX_IMAGE_PIXELS = None

In [8]:
def sizedist_image(image, pixel_length, sigma, size_thresh):
    
    image_float = img_as_float(image)

    sig=sigma/pixel_length #µm/(µm/pixel) = pixels
    minarea=size_thresh/pixel_length**2 #µm^2/(µm/pixel)^2 = pixels^2
    filtered = filters.gaussian(image_float, sigma=sig)
    mask = filtered > 0.1
    largemask = skimage.morphology.remove_small_objects(mask, min_size=minarea)
    labelmask = measure.label(largemask)

    properties = measure.regionprops(labelmask)
    area_dist = np.array([prop.area*pixel_length**2 for prop in properties]) #nm^2
    particles = len(area_dist)
    return area_dist

import os

folder=os.listdir("/Users/Addie Wu/OneDrive - UC San Diego/UCI Lab/3- Python/STORM quantification on Python/647H2A")



condswtpmb=[]
for i in folder:
    if '647' in i:
        tail=i.index('histone')
        head=i.index('')
        cond=i[head:tail]
        condswtpmb.append(cond)
condswtpmb;

number=[]
for i in condswtpmb:
    t = i[:-4]
    number.append(t)
number;


dfwtpmb=pd.DataFrame()
TotalPixVal=0
iteration=0

for t in number:
    for j in condswtpmb:
        if t in j:
            for i in mergewtpmb:
                if str(j) in i:
                    filename = i
                    image = skimage.io.imread(filename)
                    image_gray = Image.open(filename).convert('L') # convert to gray
                    image_float = img_as_float(image_gray)
                    
                    with Image.open(filename) as img:
                        meta_dict = {}
                        for key in img.tag:     
                            meta_dict[TAGS.get(key)] = img.tag[key] 

                    pixel_length = 10 #nm
                   
                    xpix = meta_dict['ImageWidth'][0]
                    ypix = meta_dict['ImageLength'][0]
                    
                    dist = []
                    sigma = 0.2
                    area_min = 1
                    
                    dist = sizedist_image(image_gray, pixel_length, sigma, area_min)
                    
                    A = pd.DataFrame(dist, columns=[t])
                    dfwtpmb = pd.concat([dfwtpmb,A], axis=1)       
                    
                    PixVal = np.sum(image_gray)
                    TotalPixVal = TotalPixVal + PixVal
                    iteration = iteration +1
                    
dfwtpmb;


histwtpmb = pd.DataFrame()

for hr in dfwtpmb.columns:
    hist, bins = np.histogram(dfwtpmb[hr], bins=range(int(minarea),30000,100))
    histS = pd.DataFrame(hist, index=bins[:-1], columns=[f'{hr}'])
    histwtpmb = pd.concat([histwtpmb,histS], axis=1)
histwtpmb.head()

histwtpmb['sum'] = histwtpmb.sum(axis = 1)
histwtpmb.head()

In [11]:
# Export DataFrame to Excel
histwtpmb.to_excel('count dataframe.xlsx', index=True)

In [ ]:
X=12  #fig width
I=1  #number of rows
N=1  #number of columns
Y=10  #fig height


fig, axes = plt.subplots(figsize=(X,Y),nrows=I,ncols=N)

plt.rcParams.update({'font.size': 25})

cellmun= iteration   # total cell numbers in all the images

axes.bar(histwtpmb.index, histwtpmb['sum']/cellmun, width=50,linewidth=1,edgecolor='black', color='lightblue') #label = 'label'

axes.legend()
axes.set_title('E. coli H2A')
axes.set_xlabel(f'Cross-sectional area ($n$m$^2$)')
axes.set_ylabel(f'Average count/cell')
